In [7]:
from PIL import Image
import os

image_dir = 'EuropeanCastles/Castle_Images'

# Walk through the directories and subdirectories
for subdir, dirs, files in os.walk(image_dir):
    for file in files:
        try:
            img = Image.open(os.path.join(subdir, file)) # open the image file
            img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
            print('Bad file:', os.path.join(subdir, file)) # print out the names of corrupt files
print("no bad files")

no bad files


In [8]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import PIL
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [9]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

print("training set: ")
train_generator = datagen.flow_from_directory(
    'EuropeanCastles/Castle_Images',
    target_size = (500,500),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training'
)

print("validation set: ")
validation_generator = datagen.flow_from_directory(
    'EuropeanCastles/Castle_Images',
    target_size = (500,500),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'validation'
)

training set: 
Found 4268 images belonging to 37 classes.
validation set: 
Found 1057 images belonging to 37 classes.


In [10]:
num_classes = 37

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(500, 500, 3)))
model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(1024, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch = None, # train_generator.samples // train_generator.batch_size,
    validation_data = validation_generator,
    validation_steps = None, epochs = 10) # validation_generator.samples // validation_generator.batch_size, epochs=20)

Epoch 1/10
 35/134 ━━━━━━━━━━━━━━━━━━━━ 9:19 6s/step - accuracy: 0.0829 - loss: 157.6797

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy : {accuracy}')

In [ ]:
img = image.load_img('EuropeanCastles/Castle_Images/Isle_of_Man/Peel_Castle/1.jpg', target_size=(500,500))

img_array = image.img_to_array(img)
img_array = img_array / 255.0

img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

predicted_class = np.argmax(prediction[0])

class_indices = train_generator.class_indices

index_to_class = {v: k for k, v in class_indices.items()}

predicted_class_name = index_to_class[predicted_class]

print('Predicted class: ', predicted_class_name)

In [ ]:
# Save model and labels

import pickle

with open('class_indices.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)

model.save('locate.h5') # h5 is legacy, if encoutering errors, save as .keras instead

In [ ]:
# load model and predict example
"""
from tensorflow.keras.models import load_model
import pickle

with open('class_indices.pkl', 'wb') as f:
    class_indices = pickle.load(f)

model = load_model('path_to_h5_file.h5')

img = image.load_img('EuropeanCastles/Castle_Images/Isle_of_Man/Peel_Castle/1.jpg', target_size=(500,500))

img_array = image.img_to_array(img)
img_array = img_array / 255.0

img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

predicted_class = np.argmax(prediction[0])

index_to_class = {v: k for k, v in class_indices.items()}

predicted_class_name = index_to_class[predicted_class]

print('Predicted class: ', predicted_class_name) 

"""